In [61]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

### read data

In [62]:
data_df = pd.read_csv('data/Reviews.csv')

In [63]:
data_df = data_df.groupby(['UserId', 'ProductId']).first()

In [64]:
scores_df = data_df['Score']

In [65]:
scores_df = scores_df.reset_index()

In [66]:
users = scores_df['UserId']
items = scores_df['ProductId']

In [67]:
users = users.astype('category')
items = items.astype('category')

In [68]:
scores = scores_df['Score']
scores = scores.astype('int32')

### create user / item sparse matrix

In [69]:
data = scores.to_numpy()
user_idx = users.cat.codes.to_numpy()
item_idx = items.cat.codes.to_numpy()

In [70]:
data.shape, user_idx.shape, item_idx.shape

((560804,), (560804,), (560804,))

In [71]:
user_names = users.cat.categories.values
item_names = items.cat.categories.values

In [72]:
users_num, items_num = user_names.shape[0], item_names.shape[0]
users_num, items_num

(256059, 74258)

In [73]:
sparse_data = csr_matrix((data, (user_idx, item_idx)),
                         shape=(users_num, items_num))

In [74]:
sparse_data

<256059x74258 sparse matrix of type '<class 'numpy.int32'>'
	with 560804 stored elements in Compressed Sparse Row format>

In [75]:
seed = 1

### split user base to train / test sets

In [76]:
train, test, train_user_names, test_user_names = train_test_split(
            sparse_data,
            user_names,
            test_size=0.1, random_state=seed, shuffle=True)

### reduce dimensions with SVD

In [77]:
svd = TruncatedSVD(n_components=100, random_state=seed)

In [78]:
train_reduced = svd.fit_transform(train)

In [79]:
train.shape

(230453, 74258)

In [80]:
train_reduced.shape

(230453, 100)

### apply nearest neighbor search using euclidean distance

In [81]:
neigh = NearestNeighbors(algorithm='ball_tree',
                         metric='minkowski',
                         # metric=cosine,
                         n_jobs=-1)

In [82]:
neigh.fit(train_reduced)

NearestNeighbors(algorithm='ball_tree', n_jobs=-1)

#### similarity depends on metric, using s = 1 / (1 + d) for euclidean

In [83]:
def get_scores(query, nn_object=neigh, n=10, sparse=train):
    """
    Retrieve n nearest neighbors from sparse matrix and calculate
    scores from averaged nonzero elements weighted with similarity
    :param query: rows from SVD-reduced array
    :param nn_object: NearestNeighbor search obj precomputed on
    SVD-reduced array
    :param n: number of neighbors to retrieve
    :param sparse: sparse matrix containing user scores
    :return: averaged scores
    """
    distance, indices = nn_object.kneighbors(query, n_neighbors=n)
    distance = distance[:, :, None]
    similarity = 1 / (1 + distance)
    neighbors = []
    for row in indices:
        row_data = sparse[row].toarray()
        neighbors.append(row_data)
    # shape: [sample_dim, neighbor_dim, item_dim]
    neighbors = np.stack(neighbors, axis=0)
    result = (neighbors * similarity).sum(axis=1)
    been_scored = np.where(neighbors != 0, 1, 0)
    norm = (been_scored * similarity).sum(axis=1)
    norm[norm == 0] = 1
    result = result / norm
    result = np.clip(result, 0, 5)  # account for float errors
    return result

test_idx = np.random.choice(train.shape[0])
test_query = train_reduced[[test_idx]]
test_scores = get_scores(test_query, n=100)
test_scores[test_scores > 0]

array([2.        , 4.33333333, 5.        , 5.        , 3.75      ,
       4.        , 3.75      , 5.        , 5.        , 5.        ,
       5.        , 2.        , 5.        , 5.        , 5.        ,
       5.        , 2.        , 4.83333333, 4.        , 5.        ,
       5.        , 5.        , 5.        , 5.        , 5.        ,
       5.        , 5.        , 5.        , 1.5       , 5.        ,
       5.        , 2.        , 5.        , 5.        , 5.        ,
       5.        , 5.        , 2.        , 5.        , 3.        ,
       5.        , 1.        , 5.        , 5.        , 5.        ,
       5.        , 5.        , 4.8       , 1.        , 5.        ,
       5.        , 5.        , 5.        , 5.        , 5.        ,
       5.        , 5.        , 5.        , 2.        , 5.        ,
       5.        , 5.        , 5.        , 5.        , 3.        ,
       5.        , 4.8       , 5.        , 5.        , 5.        ,
       5.        , 5.        , 5.        , 5.        , 5.     

In [84]:
def rmse(predicted, sparse=test):
    dense = sparse.toarray()
    mask = dense != 0
    se = np.power(predicted[mask] - dense[mask], 2)
    return np.power(se.mean(), 0.5)

In [85]:
test_reduced = svd.transform(test)

In [86]:
test_reduced.shape

(25606, 100)

In [87]:
# test size reduced due to memory limitations
# need to implement incremental calc instead
lower = 143
upper = 153
limit = slice(lower, upper, None)
test_pred = get_scores(test_reduced[limit], n=100)
error = rmse(test_pred, test[limit])

lower, upper = 40, 50
another_limit = slice(lower, upper, None)
test_pred = get_scores(test_reduced[another_limit], n=100)
another_error = rmse(test_pred, test[another_limit])

error, another_error

(0.8204670242767109, 2.875374904436263)

#### error varies strongly without large test set

In [88]:
def suggest_items(name,
                  n_items=3,
                  n_neighbors=10,
                  user_list=test_user_names,
                  item_list=item_names,
                  nn_object=neigh,
                  source_sparse=train,
                  local_sparse=test,
                  local_reduced=test_reduced):
    """
    Suggest items to user based on Nearest Neighbor search
    :param name: user name from dataset under 'UserId'
    :param n_items: number of items to suggest
    :param n_neighbors: number of neighbors to approximate scores
    :param user_list: list of user names user_id comes from
    :param item_list: list of item names from dataset under 'ProductId'
    :param nn_object: NearestNeighbor search object
    :param source_sparse: sparse data used in neighbor search
    :param local_sparse: sparse data containing user_id scores
    :param local_reduced: sparse data used to represent user_id features
    :return: suggested item indices and approximated scores
    """
    idx = np.where(user_list == name)[0][0]
    query = local_reduced[idx].reshape(1, -1)
    neighbor_scores = get_scores(query, nn_object, n_neighbors, source_sparse).reshape(-1)
    user_scores = local_sparse[idx].toarray().reshape(-1)
    neighbor_scores[user_scores > 0] = 0
    item_indices = neighbor_scores.argsort()[-n_items:]
    suggested_names = item_list[item_indices]
    suggested_scores = neighbor_scores[item_indices]
    return suggested_names, suggested_scores

In [89]:
user_id = test_user_names[43]
suggested = suggest_items(user_id, n_items=10, n_neighbors=100)

user_id, suggested

('A3UMRKK2HAUFGC',
 (array(['B0001JXBE2', 'B000LKZCPC', 'B001EQ55E0', 'B004T3312I',
         'B001ELL3LE', 'B004VDGX30', 'B000XTGCLY', 'B004M0Y8T8',
         'B001EQ5SGU', 'B0026LQW4Y'], dtype=object),
  array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])))